In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import scanpy as sc
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from tqdm import tqdm

from simple_Linear_model import simple_linear

In [3]:
# data = torch.Tensor(np.random.random((32,40)))
# t = torch.Tensor(np.ones((1,)))

model = simple_linear(input_size=50,output_size=50,hidden_size=64)

In [4]:
dataset_path = '/home/hanyuji/Results/VAE_result/DR_latent_50.pkl'

with open(dataset_path, 'rb') as f:
    data_list = pickle.load(f)

label_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
def CombinationDataloader(data_list, label_list):
    combinations = []  # 存储所有可能的三点组合
    combinations_label = []

    # 生成所有可能的三点组合
    for i in range(len(data_list)):
        for j in range(i + 1, len(data_list)):
            for k in range(j + 1, len(data_list)):
                combinations.append((data_list[i], data_list[j], data_list[k]))
                combinations_label.append((label_list[i], label_list[j], label_list[k]))

    return (combinations, combinations_label)

In [6]:
combinations, combinations_label = CombinationDataloader(data_list,label_list)

In [7]:
device=torch.device('cuda:1')
num_epoch = 30

criterion = nn.MSELoss()
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0)





In [8]:
input_size = 50
batch_size = 256

t_epoch = tqdm(range(num_epoch), ncols=100)

model.train()
for epoch in t_epoch:
    epoch_loss = 0.0
    for i, (c, l) in enumerate(zip(combinations, combinations_label)):

        c0, c1, c2 = c[0], c[1], c[2]
        
        cell_idx_0 = np.random.choice(np.arange(c0.shape[0]), size = batch_size, replace = (c0.shape[0] < batch_size))
        cell_idx_1 = np.random.choice(np.arange(c1.shape[0]), size = batch_size, replace = (c1.shape[0] < batch_size))
        cell_idx_2 = np.random.choice(np.arange(c2.shape[0]), size = batch_size, replace = (c2.shape[0] < batch_size))
        c0 = c0[cell_idx_0, :]
        c1 = c1[cell_idx_1, :]
        c2 = c2[cell_idx_2, :]
        
        
        x1, x2, x3 = (
            torch.tensor(c0).type(torch.float32).to(device),
            torch.tensor(c1).type(torch.float32).to(device),
            torch.tensor(c2).type(torch.float32).to(device),
        )
        t1, t2, t3 = (
            torch.tensor(np.full(input_size, l[0])).type(torch.float32).to(device),
            torch.tensor(np.full(input_size, l[1])).type(torch.float32).to(device),
            torch.tensor(np.full(input_size, l[2])).type(torch.float32).to(device),
        )

       
        x2_pred = model(x1, x3, t1, t2, t3)

        loss = criterion(x2, x2_pred)

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # type: ignore
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss / (i + 1)  # type: ignore

    t_epoch.set_postfix_str(f'loss:{epoch_loss:.5f}')  # type: ignore

100%|█████████████████████████████████████████████████| 30/30 [00:27<00:00,  1.10it/s, loss:0.96842]


In [ ]:

def SinkhornLoss(true_obs, est_obs, blur=0.05, scaling=0.5, batch_size=None):
    '''
    Wasserstein distance computed by Sinkhorn algorithm.
    :param true_obs (torch.FloatTensor): True expression data.
    :param est_obs (torch.FloatTensor): Predicted expression data.
    :param blur (float): Sinkhorn algorithm hyperparameter. Default as 0.05.
    :param scaling (float): Sinkhorn algorithm hyperparameter. Default as 0.5.
    :param batch_size (None or int): Either None indicates using all true cell in computation, or an integer indicates
                                     using only a batch of true cells to save computational costs.
    :return: (float) Wasserstein distance average over all timepoints.
    '''
    n_tps = len(true_obs)
    loss = 0.0
    ot_solver = geomloss.SamplesLoss("sinkhorn", p=2, blur=blur, scaling=scaling, debias=True, backend="tensorized")
    for t in range(n_tps):
        t_est = est_obs[:, t, :]
        t_true = true_obs[t]
        if batch_size is not None:
            cell_idx = np.random.choice(np.arange(t_true.shape[0]), size = batch_size, replace = (t_true.shape[0] < batch_size))
            t_true = t_true[cell_idx, :]
        t_loss = ot_solver(t_true, t_est)
        loss += t_loss
    loss = loss / n_tps
    return loss